# Aggregate Listings Data

* This notebook concatenates Airbnb listing files from http://insideairbnb.com/ and creates both wide form and long form aggregate datasets.
* The datasets are representative of a balanced panel for a given set of unique Airbnb listings. When a listing doesn't appear in a given month, it is still asssigned an entry in the dataset.

## Import and Setup

In [4]:
import os
import csv
import numpy as np
import pandas as pd
import gzip
from itertools import compress

import warnings

# Suppress innocuous warning about data fragmentation
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning) 

In [6]:
# Select city to work with

city_folder = '/united-states_new-york-city'
city_abbrev = 'NYC'

In [3]:
# Store preliminary directory, use of os should make this compatible for any user with access to the repository
cwd1 = os.getcwd() 

# Go up one directory level
os.chdir('..')
cwd2 = os.getcwd()

# Make sure repository has a 0. Raw data folder!
data_dir = cwd2 + '/0. Raw data' + city_folder

# Revert to preliminary directory
os.chdir(cwd1)

In [4]:
# This function converts values into an integer, if it fails return a string.

def IntorStr(value):
    try:
        return int(value)
    except:
        return str(value)

## Collecting listings.csv.gz files

In [5]:
# Collect the listings CSVs

numFiles = []
fileNames = os.listdir(data_dir)
for fileNames in fileNames:
    if fileNames.endswith("_listings.csv.gz"):
        numFiles.append(fileNames)
    
city = numFiles[0].split("_")[1]
print(city)

# Count the number of files
numFiles = np.sort(numFiles)
print(len(numFiles))

# Take a look at the first 5 listing files
print(numFiles[:5])

new-york-city
3
['united-states_new-york-city_2021-06-02_listings.csv.gz'
 'united-states_new-york-city_2021-09-01_listings.csv.gz'
 'united-states_new-york-city_2021-12-04_listings.csv.gz']


## Check if a file is missing specific columns
The loop below accepts a list of data file names and a list of column names and then prints if a file is missing a particular variable.

In [6]:
# First, take a look at columns in the first four files

file_vars = [] # List of all variables in a scrape month
N_file_vars = [] # Simple count of all variables in a scrape month

for my_file in numFiles:
    data = pd.read_csv(data_dir + '/' + my_file, encoding= 'iso-8859-1', low_memory = False)
    data_columns = list(data.columns)
    file_vars.append(data_columns)
    N_file_vars.append(len(data_columns))

In [7]:
# CREATE A DROP INDICATOR FOR DATA SCRAPES THAT ARE MISSING KEY VARIABLES

counter = 0
drop_list = []

for date_vars in file_vars:
    
    # I need instant bookable to be included in my set of variables, could include many more conditions here
    if 'instant_bookable' in date_vars:
        pass 
    
    else: 
        drop_list.append(counter)
    
    counter +=1
    
print("Date files to be dropped:")
print(numFiles[drop_list])

Date files to be dropped:
[]


In [8]:
# Delete date files that are missing variables of interest
numFiles = np.delete(numFiles, drop_list)

In [9]:
def check_data_cols(files, variables):
    
    """ 
    This function accepts a list of data file names (strings) 
    and a list of column names (strings) and then prints if 
    a file is missing a particular variable.
    """
    
    for my_file in files:
        data = pd.read_csv(data_dir + '/' + my_file, encoding = 'iso-8859-1', low_memory = False)
        data_columns = list(data.columns)
        
        for my_column in variables:
            if my_column not in data_columns:
                print(my_column + " missing from:")
                print(my_file)
        
        print("------")

In [10]:
# This loop is relatively slow. It takes about a minute and a half.

def find_compatible_columns(most_missing_columns):
    
    """ 
    This function accepts a filename (string). The function then 
    returns a list of the column names that exist in all files
    in numFiles based on the column names of the passed filename.
    """
    
    a = set(most_missing_columns.columns)
    
    for i in numFiles:
        data = pd.read_csv(data_dir + '/'  + i, encoding = 'iso-8859-1', low_memory = False)
        data_columns = set(data.columns)
    
        if i == numFiles[0]:
            compat = data_columns.intersection(a)
        
        compat = compat.intersection(data_columns)
    
    compat = list(compat)
    
    return compat

In [11]:
# This cell is city-specific. One needs to be careful if not using Portland! - changed to New York City


# This feeds the file with the most missing columns into the above function to create a compatible column set for all the data..
most_missing = pd.read_csv(data_dir + '/united-states_new-york-city_2021-06-02_listings.csv.gz', encoding='iso-8859-1', low_memory = False)
compatible = find_compatible_columns(most_missing)

# Make id the first cell
compatible.remove('id')
compatible.insert(0, 'id')

In [12]:
most_missing.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [2]:
# This code creates and displays a list of important variables not found in the compatible set.

airbnb_metrics = ['id', 'last_scraped', 'host_id', 'host_name', 
                  'host_since', 'host_location', 'host_response_time', 'host_response_rate',
                  'host_is_superhost', 'host_listings_count', 'host_total_listings_count', 'neighbourhood',
                  'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'street', 'zipcode', 'latitude', 
                  'longitude', 'is_location_exact', 'property_type', 'room_type', 
                  'accommodates', 'bathrooms', 'bedrooms', 'beds', 
                  'bed_type', 'square_feet', 'price', 'weekly_price',
                  'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included',
                  'extra_people', 'minimum_nights', 'maximum_nights', 'calendar_updated', 
                  'calendar_last_scraped', 'has_availability', 'availability_30', 'availability_60', 
                  'availability_90', 'availability_365', 'number_of_reviews', 'first_review', 
                  'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
                  'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 
                  'requires_license', 'license', 'instant_bookable', 'cancellation_policy',
                  'calculated_host_listings_count', 'reviews_per_month', 'amenities']

airbnb_metrics_vs_compatible = list(filter(lambda i: i not in compatible, airbnb_metrics))
airbnb_metrics_vs_compatible

NameError: name 'compatible' is not defined

In [14]:
# exlcude those columns not in the datasets
for x in airbnb_metrics_vs_compatible:
    airbnb_metrics.remove(x)

In [15]:
# Uncomment to print which datasets are missing the airbnb metrics data.
# check_data_cols(numFiles, airbnb_metrics_vs_compatible)
# none for our datasets

In [16]:
def concat_spreadsheets(concat_year):
    
    """
    This function accepts a starting and ending index of numFiles 
    as arguments and returns a cocatanated dataframe of the csv data 
    corresponding to the inputted indexes.
    """  
    yearly_numfiles_bool = []
    for filename in numFiles:
        if (filename.split('_')[2][0:4] == str(int(concat_year))):
            yearly_numfiles_bool.append(True)
        else:
            yearly_numfiles_bool.append(False)
    
    sheets_df = []
    yearly_numfiles = list(compress(numFiles, yearly_numfiles_bool))
    for filename in yearly_numfiles:
        df = pd.read_csv(data_dir + '/' + filename, index_col = None, header=0, encoding='iso-8859-1', dtype={'id': "Int64"}, 
             # some rows have weird values for the id, making them NaN
            low_memory = False)
        sheets_df.append(df)
        
    sheets_df = pd.concat(sheets_df, axis=0, ignore_index=True)
    return sheets_df

In [17]:
# Identify all unique listings ids across datasets, dropping the NAs
sheet21 = concat_spreadsheets(2021)
uniq_all = sheet21[sheet21.id.notna()].id.unique()

print(uniq_all)

<IntegerArray>
[    2595,     3831,     5121,     5136,     5178,     5203,     5803,
     6848,     6872,     6990,
 ...
 53656904, 53657155, 53658866, 53660784, 53660977, 53662330, 53662542,
 53662772, 53663081, 53665099]
Length: 45426, dtype: Int64


## Create wide form of full data

In [18]:
def wide_form(UNIQ_IDS, START, END, METRICS): 
    
    """
    This function takes a list of Airbnb unique ids as well as start
    and end indexes for said list. Then, for the selected ID's
    it returns a dataframe of the relevant data in a wide format.
    
    Note on efficiency: Still getting warning that DataFrame is highly fragmented.
                        Could still make this function a bit more efficient.
    """
    
    listing_df = pd.DataFrame(UNIQ_IDS)
    listing_df.columns = ['id']
    
    print("Number of unique listings: " + str(len(listing_df)))
    
    output_df = listing_df.copy()    
    
    date_count = START
    for i in numFiles[START:END]:
 
        # Read in gzip compressed files
        file = gzip.open(os.path.join(data_dir, i), 'rt')  
        reader = csv.reader(file, delimiter=',')
        headers = next(reader)

        bnb_metrics = METRICS
        
        d={}
        for j in bnb_metrics:
            d[str(j)+"_index"] = headers.index(j)
            # print(str(j))
            # print(d[str(j)+"_index"])

        row_values = []

        for row in reader:
            value_i = []

            for j in bnb_metrics:
                try:
                    value_j = IntorStr(row[d[str(j)+"_index"]])
                except:
                    value_j = np.NaN
                value_i.append(value_j)

            row_values.append(value_i)
        
        values_df = pd.DataFrame(row_values) # Create a dataframe for the row_values      
        values_df.columns = bnb_metrics # Set column titles
        values_df = values_df.drop_duplicates(subset='id', keep='last')

        # Merge the values with their respective id and drop duplicates
        merged_df = pd.merge(listing_df, values_df, how='outer', on='id')
        merged_df = merged_df.drop_duplicates(keep='first')   
        merged_df = merged_df.reset_index()

        for k in bnb_metrics[1:]: 
            output_df[k + str(date_count)] = merged_df[k].copy() 

        output_df['List_month'+str(date_count)] = listing_df['id'].copy().isin(np.array(values_df['id']))*1 # See if the observation is in the month data  

        date_count += 1
        
    return output_df

-----

In [19]:
# Wide format for all
wideALL = wide_form(uniq_all, 0, len(numFiles), airbnb_metrics)

Number of unique listings: 45426


In [20]:
# Save wide dataframe to local directory
wideALL.to_csv(city_abbrev + '_Data_wideALL_2021.csv.gz', 
               compression = 'gzip', index=False)

---------

# Create long form of full data and save

In [7]:
# Full long dataframe creation, Save long dataframe to local directory
mylist = ['List_month']
mylist.extend(airbnb_metrics[1:])

pd.wide_to_long(
    pd.read_csv(city_abbrev + '_Data_wideALL_2021.csv.gz',low_memory = False),
    stubnames=mylist, 
    i='id', j='month').reset_index().to_csv(city_abbrev + '_Data_longALL_2021.csv.gz', 
    compression='gzip', index=False)